In [ ]:
!pip3 install -r requirements.txt

## From the original code from Dongbin Na

#### <b>Download the Dataset</b>

* The dataset contains approximately 13,000 Korean "human face" images.
* In this dataset, all faces are cropped to a resolution of 128 X 128 pixels, although some of the original images have been high resolution.
* Each image filename indicates which family (household) number it belongs to.

In [2]:
%%capture
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EbMhBPnmIb5MutZvGicPKggBWKm5hLs0iwKfGW7_TwQIKg?download=1 -O custom_korean_family_dataset_resolution_128.zip
!unzip custom_korean_family_dataset_resolution_128.zip -d ./custom_korean_family_dataset_resolution_128

#### <b>Download the Original Datasets</b>

* <b>CelebA-HQ Dataset</b>: A dataset consisting of 30,000 "human face" images.
    * This dataset has approximately 3GB of data in size and is one of the most widely used face datasets worldwide.
* <b>CelebA-HQ Identity Dataset</b>: A dataset that provides information about <b>who the individuals (subjects) are</b> for each facial image file.
* <b>CelebA-HQ Attribute Dataset</b>: A dataset that provides attribute information for each facial image file.
    * This attribute information includes <b>gender</b>, <b>smiling</b>, and <b>age</b>, which we will be used for a multi-label task.

In [3]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/Eb37jNPPA7hHl0fmktYqcV8B-qmPLx-ZKYQ1eFk4UPBV_A?download=1 -O CelebAMask-HQ.zip
!wget https://postechackr-my.sharepoint.com/:t:/g/personal/dongbinna_postech_ac_kr/EVRoUY8_txRFv56-KWvZrksBDWbD6adkjBxwwRN7qAC6bg?download=1 -O CelebA-HQ-identity.txt
!wget https://postechackr-my.sharepoint.com/:t:/g/personal/dongbinna_postech_ac_kr/EVrdIrPOkR1OlEWBVK8lE3AB9bFh741GnKBkNgPa8trNuA?download=1 -O CelebA-HQ-attribute.txt

--2024-01-17 10:54:18--  https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/Eb37jNPPA7hHl0fmktYqcV8B-qmPLx-ZKYQ1eFk4UPBV_A?download=1
Resolving postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/dongbinna_postech_ac_kr/Documents/Research/datasets/CelebAMask-HQ/CelebAMask-HQ.zip?ga=1 [following]
--2024-01-17 10:54:18--  https://postechackr-my.sharepoint.com/personal/dongbinna_postech_ac_kr/Documents/Research/datasets/CelebAMask-HQ/CelebAMask-HQ.zip?ga=1
Reusing existing connection to postechackr-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 3153930546 (2.9G) [application/x-zip-compressed]
Saving to: ‘CelebAMask-HQ.zip’

CelebAMask-HQ.zip   100%[===================>]   2.94G   123MB/s    in

In [4]:
%%capture
!rm -rf ./CelebAMask-HQ
!unzip CelebAMask-HQ.zip

# 1. Creating a pre-trained model

In [ ]:
!CUDA_VISIBLE_DEVICES='0' python ViTMUL_MUFAC.py \
    --num_epochs 30 \
    --seeds '1' \
    --learning_rate 0.00001 \
    --momentum 0.9 \
    --model_type 'teacher' \
    --optimizer 'adamw' \
    --manual_decay True

# 2. Finetuning with baseline

In [ ]:
!CUDA_VISIBLE_DEVICES='0' python ViTMUL_MUFAC.py \
    --num_epochs 30 \
    --seeds '1' \
    --learning_rate 0.00001 \
    --momentum 0.9 \
    --model_type 'ViT-B-16' \
    --transform 2 \
    --training_type 'finetune' \
    --optimizer 'adamw' \
    --manual_decay True \
    --save_teacher False \
    --checkpoint './MUFAC_ViT-B-16_teacher_seed_1.pth'

# 3. Finetuning with DASH

In [ ]:
!CUDA_VISIBLE_DEVICES='0' python ViTMUL_MUFAC_ER.py \
    --num_epochs 20 \
    --seeds '1' \
    --learning_rate 0.00001 \
    --momentum 0.9 \
    --model_type 'ViT-B-16' \
    --transform 2 \
    --training_type 'finetune' \
    --optimizer 'adamw' \
    --manual_decay True \
    --save_teacher False \
    --checkpoint './MUFAC_ViT-B-16_teacher_seed_1.pth' \
    --DASH_layer 6